In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import m3utils
import holidays
from sklearn.ensemble import RandomForestRegressor
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) # falls plotly mal nichts anzeigt, diese Zelle wieder ausführen

# Feature Engineering für Zeitreihen
Dieses Notebook zeigt die Beispiele zum Feature Engineering für Zeitreihen. Wir nutzen dafür zuerst wieder die COVID-19-Daten.

In [ ]:
df = m3utils.load_covid_19_data()
df.sample(5)

In [ ]:
%%time
def train_predict(features):
    df = features.copy()
    df['Country/Region'] = df['Country/Region'].cat.codes
    df['date'] = m3utils.date_to_days(df['date'])
    
    result = m3utils.train_predict(df, 7, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', RandomForestRegressor(), ['date'])
    
    result['date'] = m3utils.days_to_date(result['date'])
    result['Country/Region'] = features['Country/Region'].cat.categories[result['Country/Region'].astype('int')]
    return result

m3utils.crossvalidate(df, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', train_predict, pd.to_datetime('2020-09-01'), 5, 28, m3utils.wmape)

# Features mit Zeitbezug

## Features mit Komponenten aus Datum und Zeit

Komponenten aus dem Datumswert oder Zeitwert einer Series lassen sich über den Accessor `pandas.Series.dt` ([API-Docs](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.date.html)) extrahieren.

Wild alle verfügbaren Komponenten zu extrahieren und in das ML-Modell zu packen ist aber nicht empfehlenswert. Datum und Zeit identifizieren in Kombination mit anderen Attributen oft wenige Werte und können deshalb von den ML-Modellen zum Overfitting genutzt werden. Die extrahierten Komponenten sollten deshalb immer domänenspezifisch geprüft werden: Was soll das Modell potentiell lernen?

Im COVID-19-Beispiel wäre z.B. vorstellbar, das in Abhängigkeit der Jahreszeit ein unterschiedliches Infektionsrisiko. Wir könnten dafür z.B. den Monat und das Quartal in das Modell geben (eigentlich wissen wir aber in diesem konkreten Beispiel, dass wir mit ca. 15 Monaten dafür zu wenig Daten haben).

In [ ]:
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df.sample(5)

## Features aus Ereignissen/Zeitpunkten erzeugen

### Exogene Ereignisse: Feiertage

Ein gutes Beispiel für exogene Ereignisse sind Feiertage. Dafür gibt es z.B. das Paket [Python-Holidays](https://github.com/dr-prodigy/python-holidays), das für sehr viele Länder die Feiertage berechnen kann. Wenn sich viele Menschen zu Feiertagen treffen, könnte das ein gutes Feature sein.

In [ ]:
countries = m3utils.EU_UK_data()
countries = countries.loc[countries['Country/Region'].isin(df['Country/Region'].unique())]

holidays_df = pd.DataFrame()
for i, row in countries.iterrows():
    country_holidays = holidays.CountryHoliday(row['Country_Code'], years=[2020,2021])
    holidays_df = holidays_df.append(
        pd.DataFrame({'date': country_holidays.keys(),
                      'holiday': country_holidays.values(),
                      'Country/Region': row['Country/Region']})
    )
holidays_df['date'] = pd.to_datetime(holidays_df['date'])
holidays_df['Country/Region'] = holidays_df['Country/Region'].astype(df.dtypes['Country/Region'])
holidays_df.sample(5)

#### Ereignisse als boolschen Wert kodieren

In [ ]:
df = df.drop(columns='holiday', errors='ignore').merge(holidays_df, on=['Country/Region', 'date'], how='left')
df['holiday'] = df['holiday'].notnull().astype('int')
df.loc[df['date'].isin(['2020-12-24', '2020-12-25'])]

#### Ereignisse als Zeitspanne kodieren

In [ ]:
df = df.drop(columns='holiday', errors='ignore').merge(holidays_df, on=['Country/Region', 'date'], how='left')
# Namen des Feiertags df['holiday'] durch das Datum des Feiertags ersetzen
df.loc[df['holiday'].notnull(), 'holiday'] = df.loc[df['holiday'].notnull(), 'date']
# das Feiertagsdatum nach vorne auffüllen, df['holiday'] enthält jetzt das Datum des letzten Feiertags
df['holiday'] = df.groupby(['Country/Region'])['holiday'].ffill()
# je Land und Feiertagsdatum kumuliert zählen
df['holiday'] = df.groupby(['Country/Region', 'holiday'])['holiday'].cumcount()
df.loc[(df['Country/Region']=='Germany') & (df['date'] >= '2020-12-23')].head(11)

# Metrische Features

## Basis-Features zur Beschreibung der Zeitreihe: Lags und Statistiken auf Zeitfenstern

Lags lassen sich mit der Methode [pd.Series.shift ](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.shift.html) erzeugen.

In [ ]:
max_lag = 7
lags = list(range(max_lag, 0, -1)) # oder eine eigene Liste [1, 2, 7, 14]
for lag in lags:
    df['value_daily_lag'+str(lag)] = df.groupby('Country/Region')['value_daily'].shift(lag)
df.sample(5)

Rollende Statistiken lassen sich mit den Methoden [pd.Series.rolling](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.rolling.html), [pd.Series.expanding](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.expanding.html) und [pd.Series.ewm](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.ewm.html) erzeugen.

Zum Beispiel könnten wir eine Schätzung des Pool der ansteckenden Infizierten berechnen, in dem wir annehmen, dass die Infizierten ca. 14 Tage ansteckend bleiben.

In [ ]:
df['contagious_pool'] = df.groupby('Country/Region')['value_daily'].rolling(window=14).sum().reset_index(level=0, drop=True)

Die gleitende Summe ist aber am Ende nichts anderes, als ein gleitender Mittelwert. Das ist ein Beispiel dafür, dass wir beim Feature Engineering aufpassen müssen, wie unsere Features zusammenhängen und welche ggf. redundant sind.

In [ ]:
df['contagious_pool'] = df['contagious_pool']/14

In [ ]:
fig = px.line(df,
        x='date', y=['value_daily', 'contagious_pool', 'value_trend'],
        title='Tägliche Infektionen und Anzahl der ansteckenden Infizierten',
        facet_row='Country/Region',
        labels=dict(date='Datum', value=''))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1])) # 'Italy' anstatt 'Country/Region=Italy' für die Beschriftung der Subplots
fig.update_yaxes(matches=None) # individuelle y-Achsen-Skala je Diagramm
fig.show()
df.drop(columns='contagious_pool', inplace=True)

# Interaktionen

Eine typische Interaktion zwischen zwei Variablen ist die Kombination mit Multiplikation oder Division. Viele ML-Modelle profitieren davon, da sie solche Kombinationen selbst nicht oder nur sehr aufwändig ermitteln können.

Die Reproduktionszahl (R-Wert) aus den Nachrichten ist eine solche Kombination, die sich aus dem Mittelwert der letzten 7-Tage geteilt durch den Mittelwert der letzten 7-Tage vor 4 Tagen berechnet. 

In [ ]:
df['R-number'] = df['value_trend']/(df.groupby('Country/Region')['value_trend'].shift(4)+0.001)

In [ ]:
fig = px.line(df.loc[df['date']>'2020-05-01'],
        x='date', y=['R-number'],
        title='Reproduktionszahl R',
        facet_row='Country/Region',
        labels=dict(date='Datum', value=''))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1])) # 'Italy' anstatt 'Country/Region=Italy' für die Beschriftung der Subplots
fig.update_yaxes(matches=None) # individuelle y-Achsen-Skala je Diagramm
fig.show()

In [ ]:
m3utils.crossvalidate(df, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', train_predict, pd.to_datetime('2020-09-01'), 5, 28, m3utils.wmape)

# Externe Daten

Wir fügen einen weiteren Datensatz hinzu:

Covid-19 hat bisher viel Einfluss auf das tägliche Leben, insbesondere auch auf unsere Aufenthaltsorte (Aufrufe zur Kontaktvermeidung, politische Vorgaben...). Da direkter/indirekter Kontakt für eine Infektion nötig ist, liegt es nahe Daten, die unsere Aufenthaltsorte abbilden, miteinzubeziehen. Hierfür verwenden wir Google-Bewegungsdaten (https://www.google.com/covid19/mobility/index.html?hl=de). Sie enthalten Bewegungstrends  (Referenz ist Medianwert der fünf Wochen vom 3. Januar bis 6. Februar 2020) für folgende Ortskategorien:
- `retail_and_recreation`
- `grocery_and_pharmacy`
- `parks`
- `transit_stations`
- `workplaces`
- `residential`

In den Daten selbst sind indirekt weitere Infomationen codiert (an arbeitsfreien Tagen werden die Zahlen für workplaces absinken; bei schönem Wetter werden die Zahlen für Parks steigen...).

In [ ]:
df2 = pd.read_csv('https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv', dtype={'country_region': 'category', 'metro_area': 'str', 'iso_3166_2_code': 'str'})
df2 = df2.loc[(df2['country_region'].isin(df['Country/Region'].unique())) & (df2['sub_region_1'].isnull())]
df2['date'] = pd.to_datetime(df2['date'])
df2.drop(columns=['country_region_code', 'place_id', 'sub_region_1', 'sub_region_2', 'metro_area', 'iso_3166_2_code', 'census_fips_code'], inplace=True, errors='ignore')
df2 = df2.fillna({col: 0 for col in df.columns[df.dtypes.eq(float)]}) # fehlende Werte mit 0 ersetzen
df2.rename(columns={"country_region": "Country/Region"}, inplace=True) # Spaltennamen angleichen
df2.columns = df2.columns.str.replace(r'_percent_change_from_baseline$', '', regex=True) # langes Suffix entfernen
df2['Country/Region'] = df2['Country/Region'].astype(df['Country/Region'].dtype) # Datentyp mit df angleichen für merge
df2.sample(5)

# Übung

## Aufgabe 1 - Feature Engineering für die COVID-19 Daten

Verbinde die Bewegungsdaten aus `df2` mit den anderen Daten aus `df`.

In [ ]:
# Hier dein Code für den Join

In echten Projekten solltest Du beim Feature Engineering viel visualisieren. Das lassen wir heute weg, um Zeit zu sparen :-)

Willst Du vielleicht noch andere Features erstellen? Willst Du die Bewegungsdaten so verwenden, wie sie sind, oder vielleicht auch noch andere Features auf ihnen erstellen?

In [ ]:
# Schritt 2: Zusätzliche Features?

Trainiere ein Modell mit Deinen aktuellen Features und schau, ob die Features etwas gebracht haben. Sonst wieder zu Schritt 1.

In [ ]:
# Schritt 3: ggf. Vorhersage-Code noch anpassen
m3utils.crossvalidate(df, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', train_predict, pd.to_datetime('2020-09-01'), 5, 28, m3utils.wmape)

## Aufgabe 2 - Feature Engineering mit den M5-Daten [optional]

Falls Du keine Lust mehr auf COVID-Daten hast, kannst Du auch gerne die M5-Daten kneten...

Neben den Verkaufszahlen ist eine Datei mit Preisen und ein Kalender hinterlegt. Der Kalender enthält einige Feiertage und Tage, an denen Lebensmittelmarken (SNAP) angenommen werden. Schau dir auch diese Daten dafür an.

In [ ]:
df_sales = pd.read_parquet('../input/m5-forecasting-parquet-and-aggregations/daily_sales_items_top105.parquet')
id_columns = [c for c in df_sales.columns.values if 'id' in c]
for c in id_columns:
    df_sales[c] = df_sales[c].astype('category')
df_sales.sample(5)

In [ ]:
df_prices = pd.read_parquet('../input/m5-forecasting-parquet-and-aggregations/sell_prices.parquet')
id_columns = [c for c in df_prices.columns.values if 'id' in c]
for c in id_columns:
    df_prices[c] = df_prices[c].astype('category')
df_prices.sample(5)

In [ ]:
df_calendar = pd.read_csv('../input/m5-forecasting-parquet-and-aggregations/calendar.csv')
df_calendar.sample(5)

Ein paar Inspirationen:
- Wochentag
- Anzahl Tage im Monat
- Feiertage
- Jährliche/wöchentliche Saisonalität
- Anzahl an Produkten in der Kategorie
- Trend in den Verkaufszahlen
- Trend in der Preisentwicklung
- Innerhalb der letzten 2 Wochen gab es ein Marketing-Event in der Gegend
- Verhältnis zwischen Preis und Verkaufszahl
- Regionale Unterschiede (z.B. maximaler Preis)
- Verschiedenen statistisch Werte